In [ ]:
%%bash
pip install xlsxwriter

In [ ]:
from pathlib import Path
import yaml
import pandas as pd
import networkx as nx

In [ ]:
with open("config.yaml", "r") as f:
    notebook_configuration = yaml.safe_load(f)
notebook_configuration

In [ ]:
bgcflow_dir = Path(notebook_configuration["bgcflow_dir"])
PROJECT_NAME = "TIGR03997_with_reference"
PROJECT_NAME2 = "mq_saccharopolyspora"
output_dir = Path("assets/supplementary_materials")
output_dir.mkdir(parents=True, exist_ok=True)

FIGURE_MASH = "Figure_3"

## Data S1
Data S1. Input and result tables related to the PEP on qc_saccharaopolyspora 
- Tab 1: sample.csv listing 42 genomes as input for config file
- Tab 2: NCBI metadata  
- Tab 3: CheckM results on quality assessment 
- Tab 4: SeqFu results on quality assessment 
- Tab 5: GTDB-tk and GTDB results on taxonomic definition

In [ ]:
outfile = output_dir / "DATA_S1.xlsx"

In [ ]:
PROJECT_NAME = "qc_saccharopolyspora"
PROJECT_CONFIG_DIR = bgcflow_dir / f"config/{PROJECT_NAME}"
REPORT_DIR = bgcflow_dir / f"data/processed/{PROJECT_NAME}"
df_tab1 = pd.read_csv(PROJECT_CONFIG_DIR / "samples.csv")
df_tab2 = pd.read_csv(REPORT_DIR / "tables/df_ncbi_meta.csv")
df_tab3 = pd.read_csv(REPORT_DIR / "tables/df_checkm_stats.csv")
df_tab4 = pd.read_csv(REPORT_DIR / "tables/df_seqfu_stats.csv")
df_tab5 = pd.read_csv(REPORT_DIR / "tables/df_gtdb_meta.csv").set_index("genome_id", drop=False)
df_tab5_gtdbtk = pd.read_csv(REPORT_DIR / "tables/gtdbtk.bac120.summary.tsv", sep="\t").set_index("user_genome", drop=False)

for i in df_tab5.index:
    if df_tab5.loc[i, 'Domain'] == "d__":
        for num, c in enumerate(['Domain','Phylum','Class','Order','Family','Genus', 'Organism']):
            values = df_tab5_gtdbtk.loc[i, "classification"].split(";")
            df_tab5.loc[i, c] = values[num]
    
    if df_tab5.loc[i, 'Organism'] == "s__":
        df_tab5.loc[i, "Organism"] = f"{df_tab5.loc[i, 'Genus'].strip('g__')} sp."

    if df_tab5.loc[i, 'Species'] == "s__":
        df_tab5.loc[i, "Species"] = f"{df_tab5.loc[i, 'Organism'].split()[-1]}"

In [ ]:
writer = pd.ExcelWriter(outfile, engine='xlsxwriter')
df_tab1.to_excel(writer, index=None, sheet_name='Sheet1')
df_tab2.to_excel(writer, index=None, sheet_name='Sheet2')
df_tab3.to_excel(writer, index=None, sheet_name='Sheet3')
df_tab4.to_excel(writer, index=None, sheet_name='Sheet4')
df_tab5.to_excel(writer, index=None, sheet_name='Sheet5')
writer.close()

## Data S2
Data S2. Input and result tables related to the PEP on mq_saccharaopolyspora
- Tab 1: sample.csv listing 26 genomes as input for config file
- Tab 2: MASH distan
- Tab 3: MASH Phylogroupsce
- T4b 3: Prokka result
- 5ab 4: Results of BGCs from antiSMASH
-6Tab 5: Results of GCFs based on BiG-SCAPE using different cutoffs 
ffs 


In [ ]:
outfile = output_dir / "DATA_S2.xlsx"

PROJECT_NAME = "mq_saccharopolyspora"
PROJECT_CONFIG_DIR = bgcflow_dir / f"config/{PROJECT_NAME}"
REPORT_DIR = bgcflow_dir / f"data/processed/{PROJECT_NAME}"
bigscape_dir = REPORT_DIR / "bigscape/for_cytoscape_antismash_6.1.1"

In [ ]:
df_tab1 = pd.read_csv(PROJECT_CONFIG_DIR / "samples.csv")
df_tab2 = pd.read_csv(REPORT_DIR / "mash/df_mash.csv")
df_tab3 = pd.read_csv(f"assets/tables/{FIGURE_MASH}b_mash_hcluster.csv", index_col=0)
df_tab4 = pd.read_csv("assets/tables/Figure_S8_prokka.csv", index_col=0)
df_tab4.index.name = "genome_id"
df_tab5 = pd.read_csv(REPORT_DIR / "tables/df_regions_antismash_6.1.1.csv")
df_tab6 = pd.read_csv([i for i in bigscape_dir.glob("*df_clusters_0.30*")][0])
df_tab7 = pd.read_csv([i for i in bigscape_dir.glob("*df_clusters_0.40*")][0])
df_tab8 = pd.read_csv([i for i in bigscape_dir.glob("*df_clusters_0.50*")][0])
df_tab9 = pd.read_csv([i for i in bigscape_dir.glob("*df_network_0.30*")][0])
df_tab10 = pd.read_csv([i for i in bigscape_dir.glob("*df_network_0.40*")][0])
df_tab11 = pd.read_csv([i for i in bigscape_dir.glob("*df_network_0.50*")][0])

In [ ]:
writer = pd.ExcelWriter(outfile, engine='xlsxwriter')
df_tab1.to_excel(writer, index=None, sheet_name='Sheet1')
df_tab2.to_excel(writer, index=None, sheet_name='Sheet2')
df_tab3.to_excel(writer, index=None, sheet_name='Sheet3')
df_tab4.to_excel(writer, index=None, sheet_name='Sheet4')
df_tab5.to_excel(writer, index=None, sheet_name='Sheet5')
df_tab6.to_excel(writer, index=None, sheet_name='Sheet6')
df_tab7.to_excel(writer, index=None, sheet_name='Sheet7')
df_tab8.to_excel(writer, index=None, sheet_name='Sheet8')
df_tab9.to_excel(writer, index=None, sheet_name='Sheet9')
df_tab10.to_excel(writer, index=None, sheet_name='Sheet10')
df_tab11.to_excel(writer, index=None, sheet_name='Sheet11')
writer.close()

## Data S3
Data S3. Results of BiG-FAM and ARTS database related to the PEP on mq_saccharaopolyspora 
- Tab 1: Hits against the BiG-FAM GCFs calculated using query BiG-SLICE rule
- Tab 2: List of BiG-FAM GCFs hits
- Tab 3: Hits against ARTS profiles
- Tab 4: Table with all edges represented in the enriched network (Figure 5) 5)


In [ ]:
outfile = output_dir / "DATA_S3.xlsx"

PROJECT_NAME = "mq_saccharopolyspora"
PROJECT_CONFIG_DIR = bgcflow_dir / f"config/{PROJECT_NAME}"
REPORT_DIR = bgcflow_dir / f"data/processed/{PROJECT_NAME}"
bigfam_dir = REPORT_DIR / "bigslice/query_as_6.1.1"

In [ ]:
df_tab1 = pd.read_csv(bigfam_dir / "query_network.csv")
df_tab2 = pd.read_csv("assets/tables/Figure_4_bigfam_models.csv", index_col=0)
df_tab2.index.name = "gcf_id"
df_tab3 = pd.read_csv("assets/tables/Figure_4_df_arts_hits.csv", index_col=0)
df_tab4 = pd.read_csv("assets/tables/Figure_5_raw_network.csv", index_col=0)

In [ ]:
writer = pd.ExcelWriter(outfile, engine='xlsxwriter')
df_tab1.to_excel(writer, index=None, sheet_name='Sheet1')
df_tab2.to_excel(writer, sheet_name='Sheet2')
df_tab3.to_excel(writer, index=None, sheet_name='Sheet3')
df_tab4.to_excel(writer, index=None, sheet_name='Sheet4')
writer.close()

## Data S4
Data S4. Results of detailed GCF comparison related to the PEP on staphylobactin-like BGCs
- Tab 1: COG table
- Tab 2: Clinker links for BGC alignment of staphylobactin-like GCF
- Tab 3 CBlaster hits for BGC alignment of staphylobactin-like GCFCFre 5)

In [ ]:
outfile = output_dir / "DATA_S4.xlsx"

PROJECT_NAME = "Staphylobactin"
FIGURE = "Figure_S11"
PROJECT_CONFIG_DIR = bgcflow_dir / f"config/{PROJECT_NAME}"
REPORT_DIR = bgcflow_dir / f"data/processed/{PROJECT_NAME}"

In [ ]:
df_tab1 = pd.read_csv(f"assets/tables/{FIGURE}_{PROJECT_NAME}_gggenomes_feat.csv")
df_tab2 = pd.read_csv(f"assets/tables/{FIGURE}_{PROJECT_NAME}_clinker_links.csv")
df_tab3 = pd.read_csv(f"assets/tables/{FIGURE}_cblaster_{PROJECT_NAME}_blast_hits.csv")

In [ ]:
writer = pd.ExcelWriter(outfile, engine='xlsxwriter')
df_tab1.to_excel(writer, index=None, sheet_name='Sheet1')
df_tab2.to_excel(writer, index=None, sheet_name='Sheet2')
df_tab3.to_excel(writer, index=None, sheet_name='Sheet3')
writer.close()

## Data S5

Data S5. Results of detailed GCF comparison related to the PEP on spinosyn-like BGCs
- Tab 1: COG table
- Tab 2: Clinker links for BGC alignment of spinosyn-like GCF
- Tab 3 CBlaster hits for BGC alignment of spinosyn-like GCFCF


In [ ]:
outfile = output_dir / "DATA_S5.xlsx"

PROJECT_NAME = "Spinosyn"
FIGURE = "Figure_S10"
PROJECT_CONFIG_DIR = bgcflow_dir / f"config/{PROJECT_NAME}"
REPORT_DIR = bgcflow_dir / f"data/processed/{PROJECT_NAME}"

In [ ]:
df_tab1 = pd.read_csv(f"assets/tables/{FIGURE}_{PROJECT_NAME}_gggenomes_feat.csv")
df_tab2 = pd.read_csv(f"assets/tables/{FIGURE}_{PROJECT_NAME}_clinker_links.csv")
df_tab3 = pd.read_csv(f"assets/tables/{FIGURE}_cblaster_{PROJECT_NAME}_blast_hits.csv")

In [ ]:
writer = pd.ExcelWriter(outfile, engine='xlsxwriter')
df_tab1.to_excel(writer, index=None, sheet_name='Sheet1')
df_tab2.to_excel(writer, index=None, sheet_name='Sheet2')
df_tab3.to_excel(writer, index=None, sheet_name='Sheet3')
writer.close()

## Data S6

Data S6. Results of detailed GCF comparison related to the PEP on Erythreapeptin-like BGCs
- Tab 1: COG table
- Tab 2: Clinker links for BGC alignment of Erythreapeptin-like GCF
- Tab 3 CBlaster hits for BGC alignment of Erythreapeptin-like GCF

In [ ]:
outfile = output_dir / "DATA_S6.xlsx"

PROJECT_NAME = "Erythreapeptin"
FIGURE = "Figure_6b"
PROJECT_CONFIG_DIR = bgcflow_dir / f"config/{PROJECT_NAME}"
REPORT_DIR = bgcflow_dir / f"data/processed/{PROJECT_NAME}"

In [ ]:
df_tab1 = pd.read_csv(f"assets/tables/{FIGURE}_{PROJECT_NAME}_gggenomes_feat.csv")
df_tab2 = pd.read_csv(f"assets/tables/{FIGURE}_{PROJECT_NAME}_clinker_links.csv")
df_tab3 = pd.read_csv(f"assets/tables/{FIGURE}_cblaster_{PROJECT_NAME}_blast_hits.csv")

In [ ]:
writer = pd.ExcelWriter(outfile, engine='xlsxwriter')
df_tab1.to_excel(writer, index=None, sheet_name='Sheet1')
df_tab2.to_excel(writer, index=None, sheet_name='Sheet2')
df_tab3.to_excel(writer, index=None, sheet_name='Sheet3')
writer.close()

## Data S7

Data S7. Results of detailed GCF comparison related to the PEP on Mycofactocin-like BGCs
- Tab 1: COG table
- Tab 2: Clinker links for BGC alignment of Mycofactocin-like GCF
- Tab 3 CBlaster hits for BGC alignment of Mycofactocin-like GCF

In [ ]:
outfile = output_dir / "DATA_S7.xlsx"

PROJECT_NAME = "TIGR03997_with_reference"
FIGURE = "Figure_7b"
PROJECT_CONFIG_DIR = bgcflow_dir / f"config/{PROJECT_NAME}"
REPORT_DIR = bgcflow_dir / f"data/processed/{PROJECT_NAME}"

In [ ]:
df_tab1 = pd.read_csv(f"assets/tables/{FIGURE}_{PROJECT_NAME}_gggenomes_feat.csv")
df_tab2 = pd.read_csv(f"assets/tables/{FIGURE}_{PROJECT_NAME}_clinker_links.csv")
df_tab3 = pd.read_csv(f"assets/tables/{FIGURE}_cblaster_{PROJECT_NAME}_blast_hits.csv")

In [ ]:
writer = pd.ExcelWriter(outfile, engine='xlsxwriter')
df_tab1.to_excel(writer, index=None, sheet_name='Sheet1')
df_tab2.to_excel(writer, index=None, sheet_name='Sheet2')
df_tab3.to_excel(writer, index=None, sheet_name='Sheet3')
writer.close()